In [11]:
%load_ext autoreload
%autoreload 2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,OneHotEncoder
from xgboost import XGBClassifier, XGBRegressor
import pathlib
import glob
from sklearn.metrics import accuracy_score,precision_score,median_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pickle import dump,load

dir_path = pathlib.Path('htmlData/preprocessed/')
csv_path = dir_path.joinpath('*.csv')


all_files = glob.glob(csv_path.as_posix())
df=None
for f in all_files:
    if  df is None:
        df = pd.read_csv(f)
    else:
        df = pd.concat([df,pd.read_csv(f)])
# df = df.drop(['market cap','volume','operating cash flow'],axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# With all features
numerical_cols = df.select_dtypes('number').columns
id_feature = ['Stock_code']
categorical_cols = df.select_dtypes(exclude = ['number','boolean']).columns.drop(['Stock_code'])
# target_feature = ['trend_sp']

X = df.drop( id_feature,axis=1)
# y = df[target_feature]

In [14]:
preprocessor = load(open('model/preprocessor.pkl', 'rb'))
X_test = preprocessor.transform(X)

In [15]:
model = XGBRegressor()
model.load_model('model/xgboost_0311.json')

y_test_hat = model.predict(X_test)

In [16]:
k=30

topK = df.iloc[y_test_hat.argsort()[-k:]]

In [18]:
print(topK)

     Stock_code  market_cap  price_per_sale  current_price  price_per_book  \
5582    TNEX.OB        1860            1.20         0.5100             NaN   
1160    CLAR.OB        2220            0.02         0.0490             NaN   
512     DLTA.OB        2120            1.57         0.1500            1.04   
313        ANGN       24650            0.88         6.7900            1.95   
2281       FTGX       18100            0.48         2.8700            0.38   
3441    MAMM.OB         540            1.64         0.1100             NaN   
4572    PSCO.OB         486            0.21         0.0600             NaN   
1442       CTIB        7530            0.25         3.5500            2.04   
1564    SYEV.PK       13230            2.26         0.5100            4.71   
3415    MAMM.OB         295            0.90         0.0600             NaN   
4277       PARD       98020         7540.22         4.3000            1.73   
920     JUHL.OB       11030            0.94         0.5000      

In [77]:
topK.Stock_code.to_csv('predictions/pred.csv',index=False,header=False)